In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install advertools
import advertools as adv
import pandas as pd
import regex as re
import ast

In [ ]:
import pandas as pd
import regex as re
import ast

### Using networkx


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/tweetsconcat.csv')
df = df.drop(['index'], axis=1)
df

In [ ]:
import networkx as nx

In [ ]:
df2 = df["author"].astype('str')
df2 = df2.apply(lambda x: ast.literal_eval(x))
df2 = df2.apply(pd.Series)
df['username'] = pd.Series(df2['username'])
df

,entities,context_annotations,source,attachments,possibly_sensitive,text,public_metrics,reply_settings,id,lang,...,created_at,author_id,author,__twarc,referenced_tweets,in_reply_to_user_id,in_reply_to_user,geo,withheld,username
0,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344769054859534337'], 'med...",False,How do you keep an almost-eradicated virus in ...,"{'retweet_count': 36, 'reply_count': 19, 'like...",everyone,1344769057720070144,en,...,2020-12-31 22:15:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
1,"{'hashtags': [{'start': 54, 'end': 62, 'tag': ...","[{'domain': {'id': '65', 'name': 'Interests an...",Sprout Social,"{'media_keys': ['3_1344760747851657219'], 'med...",False,Attending a small celebration? Take steps to m...,"{'retweet_count': 104, 'reply_count': 69, 'lik...",everyone,1344760752587071488,en,...,2020-12-31 21:42:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
2,"{'hashtags': [{'start': 44, 'end': 52, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344741623805173764'], 'med...",False,An ongoing CDC study of household spread of #C...,"{'retweet_count': 68, 'reply_count': 33, 'like...",everyone,1344741626430816256,en,...,2020-12-31 20:26:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
3,"{'hashtags': [{'start': 126, 'end': 134, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter Media Studio,"{'media_keys': ['13_1344705124866916352'], 'me...",False,How do we know if vaccines are safe? @aasian a...,"{'retweet_count': 38, 'reply_count': 34, 'like...",everyone,1344735073753387008,en,...,2020-12-31 20:00:00+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
4,"{'hashtags': [{'start': 38, 'end': 46, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344725018392076288'], 'med...",False,New @CDCMMWR illustrates data showing #COVID19...,"{'retweet_count': 161, 'reply_count': 108, 'li...",everyone,1344725021198065664,en,...,2020-12-31 19:20:03+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233289,"{'annotations': [{'start': 81, 'end': 83, 'pro...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,NaN,False,The amount of people not wearing masks inside ...,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",everyone,1477069420358877184,en,...,2022-01-01 00:09:27+00:00,61066540,{'description': 'pediatric nurse. capricorn st...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,"{'place_id': '895f19a3c08d3d35', 'place_type':...",NaN,kaaaramarie
233290,"{'annotations': [{'start': 126, 'end': 128, 'p...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for iPhone,NaN,False,@DCOstrander74 @IssuesOfMyTime @TeacherBW @OhN...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,1477068786117197824,en,...,2022-01-01 00:06:55+00:00,1043938043844055040,{'description': 'Virginia Tech NHL Politics NA...,{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'type': 'replied_to', 'id': '147704417537165...",7.237729e+17,"{'description': '', 'profile_image_url': 'http...","{'place_id': '3f7a925ec706ea48', 'place_type':...",NaN,KevinMeador4
233291,"{'annotations': [{'start': 28, 'end': 55, 'pro...",NaN,Twitter for iPhone,NaN,False,BREAKING: the CDC says that Hexachordal Combin...,"{'retweet_count': 1, 'reply_count': 

In [ ]:
# create a networkx directed graph
G_mention = nx.DiGraph()

# loop through each row
for r in df.iterrows():
    author = r[1]['username']
    author = f'@{author}'

    text = r[1]['text']

    # use regular expressions to extract mentions
    mentions = set(re.findall(r"@(\w+)", text))

    # add the users if there are any mentioned in the text.
    has_users = len(mentions) > 0

    if has_users:
       for u in mentions:
          u = f'@{u}'
          if G_mention.has_edge(author, u): # we added this one before, just increase the weight by one
            G_mention[author][u]['weight'] += 1
          else: # new edge. add with weight=1
             G_mention.add_edge(author, u, weight=1)

In [ ]:
#Only including links of weight >= 3:
to_be_removed = [x for  x in G_mention.nodes() if G_mention.degree(x) < 3]

for x in to_be_removed:
    G_mention.remove_node(x)

In [ ]:
df_mention = nx.to_pandas_edgelist(G_mention)

In [ ]:
df_mention = df_mention[df_mention['source']!=df_mention['target']] #removing loops

In [ ]:
df_mention

,source,target,weight
0,@CDCgov,@CDCMMWR,600
1,@CDCgov,@CDCFound,28
2,@CDCgov,@AmerMedicalAssn,5
3,@CDCgov,@AdCouncil,5
4,@CDCgov,@US_FDA,30
...,...,...,...
122665,@deniseparrish01,@cdc,1
122666,@deniseparrish01,@true_sea,1
122667,@georgesray,@TitosVodka,1
122668,@LeslieStern12,@zachlanebryan,1


In [ ]:
df_mention.to_csv('ergm_links_networkx_3_edges_minimum.csv')

### Extracting links between users (without networkx)

N.B.: this way produces the same as networkx but takes much longer, hence, stick to networkx

In [ ]:
df = pd.read_csv('tweetsconcat.csv')
df = df.drop(['index'], axis=1)

mention_summary = adv.extract_mentions(df['text'])
df =  (df.assign(mentions=mention_summary['mentions']))
df = df[df['mentions'].map(lambda d: len(d)) > 0]
df = df.reset_index(drop=True)

df2 = df["author"].astype('str')
df2 = df2.apply(lambda x: ast.literal_eval(x))
df2 = df2.apply(pd.Series)
df['username'] = pd.Series(df2['username'])
df['username'] = '@' + df['username'].astype(str)

mentionsdf1 = pd.DataFrame(columns = ["Source","Target"])
# loop through each row
for r in df.iterrows():
  author = r[1]['username']
  author = f'{author}'

  text = r[1]['text']

  count = 0
  # use regular expressions to extract mentions
  mentions1 = set(re.findall(r"@(\w+)", text))

  # add the users if there are any mentioned in the text.
  has_users = len(mentions1) > 0
  if has_users:
    for u in mentions1:
        u = f'@{u}'
        newrow = {'Source':author, 'Target':u}
        pd.concat([mentionsdf1, pd.DataFrame([newrow])], ignore_index=True)

mentionsdf1.to_csv('mentions.csv')
mentionsdf1["Count"] = 1

In [ ]:
mentionsdf1 = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/mentions.csv')

In [ ]:
mentionsdf1 = mentionsdf1[['Source','Target']]
mentionsdf1

,Source,Target
0,@CDCgov,@CDCMWMR
1,@CDCgov,@aasian
2,@CDCgov,@CDCMMWR
3,@CDCgov,@CDCMMWR
4,@CDCgov,@CDCFound
...,...,...
441886,@KevinMeador4,@DCOstrander74
441887,@KevinMeador4,@TeacherBW
441888,@KevinMeador4,@GovRonDeSantis
441889,@KevinMeador4,@FoxNews


In [ ]:
mentionsdf1['Count'] = 1
mentionsdf1

In [ ]:
df_mention2 = mentionsdf1.groupby(["Source","Target"], as_index=False)['Count'].agg('sum')
df_mention2 = df_mention2.sort_values(by='Count', ascending=False)
df_mention2 = df_mention2.reset_index(drop=True)

In [ ]:
df_mention2.to_csv('ergm_links.csv')

### Extracting nodes and their features:

In [ ]:
df0 = pd.read_json('/content/drive/MyDrive/Research-Dr.Ammari/topic_tweets/cdc_twitter_chunk_0_.zip', compression='zip')
df1 = pd.read_json('/content/drive/MyDrive/Research-Dr.Ammari/topic_tweets/cdc_twitter_chunk_1_.zip', compression='zip')
df2 = pd.read_json('/content/drive/MyDrive/Research-Dr.Ammari/topic_tweets/cdc_twitter_chunk_2_.zip', compression='zip')
df3 = pd.read_json('/content/drive/MyDrive/Research-Dr.Ammari/topic_tweets/cdc_twitter_chunk_3_.zip', compression='zip')
df4 = pd.read_json('/content/drive/MyDrive/Research-Dr.Ammari/topic_tweets/cdc_twitter_chunk_4_.zip', compression='zip')
pdList = [df0, df1, df2, df3, df4]
df = pd.concat(pdList)
df = df.reset_index(drop=True)

In [ ]:
df.head()

,entities,in_reply_to_user_id,reply_settings,possibly_sensitive,geo,text,lang,context_annotations,id,author_id,...,62,63,64,65,66,67,68,69,70,top_topic
0,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': '65e3a577f1940550', 'country': 'V...",@CDCgov please do the honorable and right thin...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1333114613177716736,18493517,...,2.415000e-07,2.423000e-07,2.352000e-07,2.524000e-07,1.921000e-07,3.026000e-07,1.887000e-07,2.835000e-07,2.135000e-07,0
1,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': '69d2c4f187635997', 'country': 'V...","@CDCgov Nobody fucking cares anymore, and a va...",en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1332930628979884032,1151963845990047744,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0
2,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': '96683cc9126741d1', 'country': 'V...",@CDCgov @cnnbrk \nKern Valley State Prison lea...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1332858032120877056,1282704419944112128,...,8.428300e-06,7.521800e-06,5.388000e-07,1.954800e-06,2.006000e-06,1.915700e-06,1.164151e-03,1.152900e-06,1.634600e-06,24
3,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': 'fbd6d2f5a4e4a15e', 'country': 'V...",@CDCgov Why are the spreading #COVIDAlert at t...,en,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1332852410415521792,1282704419944112128,...,5.750000e-08,5.150000e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.315721e-02,0.000000e+00,0.000000e+00,1
4,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': 'a612c69b44b2e5da', 'country': 'V...",@CDCgov Isn’t it wonderful that we have a 99.9...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1332798705066332160,52765914,...,7.000000e-10,7.000000e-10,7.000000e-10,7.000000e-10,6.000000e-10,8.000000e-10,5.000000e-10,8.000000e-10,6.000000e-10,0


In [ ]:
df2 = df["author"].astype('str')
df2 = df2.apply(lambda x: ast.literal_eval(x))
df2 = df2.apply(pd.Series)
df['username'] = pd.Series(df2['username'])
df['username'] = '@' + df['username'].astype(str)

In [ ]:
df.head()

,entities,in_reply_to_user_id,reply_settings,possibly_sensitive,geo,text,lang,context_annotations,id,author_id,...,63,64,65,66,67,68,69,70,top_topic,username
0,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': '65e3a577f1940550', 'country': 'V...",@CDCgov please do the honorable and right thin...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1333114613177716736,18493517,...,2.423000e-07,2.352000e-07,2.524000e-07,1.921000e-07,3.026000e-07,1.887000e-07,2.835000e-07,2.135000e-07,0,@iChrisg2009
1,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': '69d2c4f187635997', 'country': 'V...","@CDCgov Nobody fucking cares anymore, and a va...",en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1332930628979884032,1151963845990047744,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,@NathanielDWill2
2,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': '96683cc9126741d1', 'country': 'V...",@CDCgov @cnnbrk \nKern Valley State Prison lea...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1332858032120877056,1282704419944112128,...,7.521800e-06,5.388000e-07,1.954800e-06,2.006000e-06,1.915700e-06,1.164151e-03,1.152900e-06,1.634600e-06,24,@DreaminNirvana
3,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': 'fbd6d2f5a4e4a15e', 'country': 'V...",@CDCgov Why are the spreading #COVIDAlert at t...,en,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",1332852410415521792,1282704419944112128,...,5.150000e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.315721e-02,0.000000e+00,0.000000e+00,1,@DreaminNirvana
4,"{'mentions': [{'start': 0, 'end': 7, 'username...",146569971.0,everyone,False,"{'place_id': 'a612c69b44b2e5da', 'country': 'V...",@CDCgov Isn’t it wonderful that we have a 99.9...,en,"[{'domain': {'id': '123', 'name': 'Ongoing New...",1332798705066332160,52765914,...,7.000000e-10,7.000000e-10,7.000000e-10,6.000000e-10,8.000000e-10,5.000000e-10,8.000000e-10,6.000000e-10,0,@anniecollins10


In [ ]:
dfnew = df[['username','0', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41',
       '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53',
       '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65',
       '66', '67', '68', '69', '70']]

In [ ]:
dfnew = dfnew.groupby('username').mean().reset_index()

In [ ]:
dfnew.to_csv('topic_averages.csv')

In [ ]:
df_averages = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/topic_averages.csv')
df_averages = df_averages.drop(['Unnamed: 0'], axis =1)
df_averages

,username,0,1,2,3,4,5,6,7,8,...,61,62,63,64,65,66,67,68,69,70
0,@000Dave,2.477652e-17,1.110000e-08,2.289738e-17,3.604591e-16,2.239824e-17,1.220000e-08,5.851894e-17,0.000000e+00,1.370000e-08,...,9.052112e-01,0.000000e+00,0.000000e+00,3.427804e-17,3.732863e-16,4.130110e-16,3.358374e-16,1.560000e-08,8.013990e-17,1.967385e-16
1,@007_bh,2.933674e-90,5.766793e-03,2.676238e-90,4.630324e-86,2.625274e-90,5.885307e-03,1.250759e-89,4.596210e-03,8.020067e-03,...,7.820867e-03,5.773691e-03,5.051357e-03,4.587438e-90,4.732033e-86,5.807895e-86,4.335645e-86,6.844799e-03,2.888859e-89,6.892561e-88
2,@007_mz,7.077135e-33,4.366731e-24,6.312552e-33,1.611216e-01,6.220860e-33,4.051868e-24,5.477064e-32,3.568840e-24,5.348561e-24,...,5.912806e-24,4.792712e-24,4.080704e-24,1.241732e-32,1.647938e-01,1.400284e-02,1.504492e-01,4.935309e-24,3.390699e-31,3.533119e-27
3,@00micki,4.273789e-17,0.000000e+00,3.219778e-17,0.000000e+00,3.312958e-17,0.000000e+00,5.738249e-16,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,6.898132e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.972303e-01
4,@01062021AD,6.360321e-33,6.469902e-03,4.996470e-33,6.728495e-32,5.083096e-33,7.056602e-03,1.462073e-32,2.430080e-28,1.073166e-02,...,7.848185e-03,1.456592e-30,1.336663e-30,8.771123e-33,6.285132e-32,7.821660e-32,6.371423e-32,2.872893e-01,1.907144e-32,4.320233e-32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83252,@zygspytz,3.301093e-21,9.000000e-10,3.093132e-21,2.207365e-20,3.006408e-21,4.530573e-03,6.061859e-21,1.147610e-18,4.963284e-03,...,1.385749e-16,1.882815e-19,1.692350e-19,4.139036e-21,2.324945e-20,2.180580e-20,2.069214e-20,0.000000e+00,8.295328e-21,1.386862e-20
83253,@zytron,1.491000e-06,9.592100e-06,1.402700e-06,2.900500e-06,1.365900e-06,1.879670e-05,2.330700e-06,4.583900e-06,1.693700e-05,...,1.062970e-05,4.643700e-06,4.219700e-06,1.912400e-06,2.968700e-06,3.773100e-06,2.690600e-06,1.130210e-05,2.411900e-06,3.549600e-06
83254,@zzachammer,5.945000e-07,8.962000e-07,4.815000e-07,9.999402e-01,4.864000e-07,7.840000e-07,5.371000e-07,7.922000e-07,7.616000e-07,...,8.290000e-07,1.159300e-06,1.234500e-06,5.782000e-07,2.804200e-06,6.944000e-07,4.174300e-06,7.330000e-07,9.000000e-07,7.866000e-07
83255,@zzdesign,6.845564e-03,4.011482e-03,1.354386e-02,4.535548e-03,4.816095e-02,3.360061e-03,3.550069e-03,3.702906e-03,3.438196e-03,...,3.615186e-03,4.279931e-03,4.485606e-03,3.622084e-03,4.582441e-03,3.290841e-03,4.841338e-03,3.390397e-03,4.232845e-03,3.540550e-03


In [ ]:
df2 = df["author"].astype('str')
df2 = df2.apply(lambda x: ast.literal_eval(x))
df2 = df2.apply(pd.Series)

df['verified'] = pd.Series(df2['verified'])
df['username'] = pd.Series(df2['username'])
df['username'] = '@' + df['username'].astype(str)
df

,index,entities,context_annotations,source,attachments,possibly_sensitive,text,public_metrics,reply_settings,id,...,author_id,author,__twarc,referenced_tweets,in_reply_to_user_id,in_reply_to_user,geo,withheld,verified,username
0,0,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344769054859534337'], 'med...",False,How do you keep an almost-eradicated virus in ...,"{'retweet_count': 36, 'reply_count': 19, 'like...",everyone,1344769057720070144,...,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,True,@CDCgov
1,1,"{'hashtags': [{'start': 54, 'end': 62, 'tag': ...","[{'domain': {'id': '65', 'name': 'Interests an...",Sprout Social,"{'media_keys': ['3_1344760747851657219'], 'med...",False,Attending a small celebration? Take steps to m...,"{'retweet_count': 104, 'reply_count': 69, 'lik...",everyone,1344760752587071488,...,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,True,@CDCgov
2,2,"{'hashtags': [{'start': 44, 'end': 52, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344741623805173764'], 'med...",False,An ongoing CDC study of household spread of #C...,"{'retweet_count': 68, 'reply_count': 33, 'like...",everyone,1344741626430816256,...,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,True,@CDCgov
3,3,"{'hashtags': [{'start': 126, 'end': 134, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter Media Studio,"{'media_keys': ['13_1344705124866916352'], 'me...",False,How do we know if vaccines are safe? @aasian a...,"{'retweet_count': 38, 'reply_count': 34, 'like...",everyone,1344735073753387008,...,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,True,@CDCgov
4,4,"{'hashtags': [{'start': 38, 'end': 46, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344725018392076288'], 'med...",False,New @CDCMMWR illustrates data showing #COVID19...,"{'retweet_count': 161, 'reply_count': 108, 'li...",everyone,1344725021198065664,...,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,True,@CDCgov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233289,162290,"{'annotations': [{'start': 81, 'end': 83, 'pro...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,NaN,False,The amount of people not wearing masks inside ...,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",everyone,1477069420358877184,...,61066540,{'description': 'pediatric nurse. capricorn st...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,"{'place_id': '895f19a3c08d3d35', 'place_type':...",NaN,False,@kaaaramarie
233290,162291,"{'annotations': [{'start': 126, 'end': 128, 'p...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for iPhone,NaN,False,@DCOstrander74 @IssuesOfMyTime @TeacherBW @OhN...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,1477068786117197824,...,1043938043844055040,{'description': 'Virginia Tech NHL Politics NA...,{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'type': 'replied_to', 'id': '147704417537165...",7.237729e+17,"{'description': '', 'profile_image_url': 'http...","{'place_id': '3f7a925ec706ea48', 'place_type':...",NaN,False,@KevinMeador4
233291,162292,"{'annotations': [{'start': 28, 'end': 55, 'pro...",NaN,Twitter for iPhone,NaN,False,BREAKING: the CDC says that Hexachordal Combin...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",everyone,1477068540599484416,...,3404434271,{'description': 'UNL ‘22 Piano Performance | h...,{'url': 'https://api

In [ ]:
df_limited = df[['username','verified']]
df_limited = df_limited.drop_duplicates(subset=['username'])
df_limited

,username,verified
0,@CDCgov,True
7322,@NBSaphierMD,True
7323,@thenameis_ari,False
7324,@hillsy191,False
7325,@Itsmoosej,False
...,...,...
233281,@SerraJovenich,False
233282,@rodsoutherland,False
233288,@THEtacotina,False
233289,@kaaaramarie,False


In [ ]:
s = df_limited.verified.replace({False: 0, True: 1})
df_limited['verified.type'] = s

<ipython-input-21-c0a431f6d0e2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limited['verified.type'] = s


In [ ]:
df_limited = df_limited.rename({'verified': 'verified.label'}, axis='columns')

In [ ]:
df_limited['verified.label'] = df_limited['verified.label'].replace({False: 'unverified', True: 'verified'})

In [ ]:
df_limited = df_limited[['username', 'verified.type', 'verified.label']]
df_limited

,username,verified.type,verified.label
0,@CDCgov,1,verified
7322,@NBSaphierMD,1,verified
7323,@thenameis_ari,0,unverified
7324,@hillsy191,0,unverified
7325,@Itsmoosej,0,unverified
...,...,...,...
233281,@SerraJovenich,0,unverified
233282,@rodsoutherland,0,unverified
233288,@THEtacotina,0,unverified
233289,@kaaaramarie,0,unverified


In [ ]:
ergm_nodes = pd.merge(df_limited, df_averages, on="username")

In [ ]:
# limiting to mask related topics only -> 2, 4, 13, 26, 54
ergm_nodes = ergm_nodes[['username','verified.type','verified.label','2','4','13','26','54']]
ergm_nodes

,username,verified.type,verified.label,2,4,13,26,54
0,@CDCgov,1,verified,4.235155e-03,2.144329e-03,1.112680e-02,1.374142e-03,4.125985e-03
1,@NBSaphierMD,1,verified,6.021439e-02,2.844062e-02,1.009201e-03,7.112913e-04,7.714173e-04
2,@thenameis_ari,0,unverified,5.309836e-04,5.173393e-04,5.290470e-04,5.288367e-04,5.045694e-04
3,@hillsy191,0,unverified,4.583740e-02,1.582431e-02,1.427712e-03,1.382988e-02,1.253511e-03
4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...
83252,@SerraJovenich,0,unverified,1.720840e-03,1.654831e-03,1.704416e-03,1.752056e-03,1.613899e-03
83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48
83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
#ergm_nodes.to_csv('ergm_nodes_masks.csv')

In [ ]:
ergm_nodes = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/ergm_nodes_masks.csv')

In [ ]:
ergm_nodes = ergm_nodes.drop(['Unnamed: 0'], axis=1)
ergm_nodes

,username,verified.type,verified.label,2,4,13,26,54
0,@CDCgov,1,verified,4.235155e-03,2.144329e-03,1.112680e-02,1.374142e-03,4.125985e-03
1,@NBSaphierMD,1,verified,6.021439e-02,2.844062e-02,1.009201e-03,7.112913e-04,7.714173e-04
2,@thenameis_ari,0,unverified,5.309836e-04,5.173393e-04,5.290470e-04,5.288367e-04,5.045694e-04
3,@hillsy191,0,unverified,4.583740e-02,1.582431e-02,1.427712e-03,1.382988e-02,1.253511e-03
4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...
83252,@SerraJovenich,0,unverified,1.720840e-03,1.654831e-03,1.704416e-03,1.752056e-03,1.613899e-03
83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48
83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
source = list(df_mention['source'].unique())
target = list(df_mention['target'].unique())
allusers = source + target
ergm_nodes = ergm_nodes[ergm_nodes['username'].isin(allusers)]
ergm_nodes

In [ ]:
ergm_nodes.to_csv('ergm_nodes_limited.csv')

In [ ]:
ergm_nodes = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/ergm_nodes_limited.csv')
ergm_nodes

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54
0,0,@CDCgov,1,verified,0.004235,0.002144,0.011127,0.001374,0.004126
1,1,@NBSaphierMD,1,verified,0.060214,0.028441,0.001009,0.000711,0.000771
2,3,@hillsy191,0,unverified,0.045837,0.015824,0.001428,0.013830,0.001254
3,5,@cimtrbl2,0,unverified,0.000527,0.000528,0.015970,0.000521,0.030139
4,6,@JMCRHealthDept,1,verified,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...,...,...,...,...
23834,83077,@georgesray,0,unverified,0.001078,0.001048,0.001071,0.001050,0.001014
23835,83121,@middlefischer,0,unverified,0.001849,0.002175,0.002017,0.001905,0.001786
23836,83130,@deniseparrish01,0,unverified,0.527144,0.011288,0.118513,0.003220,0.058992
23837,83172,@LeslieStern12,0,unverified,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
ergm_nodes = ergm_nodes.drop(['Unnamed: 0', 'verified.label'], axis=1)
ergm_nodes

,username,verified.type,2,4,13,26,54
0,@CDCgov,1,0.004235,0.002144,0.011127,0.001374,0.004126
1,@NBSaphierMD,1,0.060214,0.028441,0.001009,0.000711,0.000771
2,@hillsy191,0,0.045837,0.015824,0.001428,0.013830,0.001254
3,@cimtrbl2,0,0.000527,0.000528,0.015970,0.000521,0.030139
4,@JMCRHealthDept,1,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...,...,...
23834,@georgesray,0,0.001078,0.001048,0.001071,0.001050,0.001014
23835,@middlefischer,0,0.001849,0.002175,0.002017,0.001905,0.001786
23836,@deniseparrish01,0,0.527144,0.011288,0.118513,0.003220,0.058992
23837,@LeslieStern12,0,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
ergm_nodes["2Q"] = pd.qcut(ergm_nodes["2"], q=[0, 0.25, 0.5, 0.75, 1.0], labels=["Q1", "Q2", "Q3", "Q4"])

In [ ]:
ergm_nodes["4Q"] = pd.qcut(ergm_nodes["4"], q=[0, 0.25, 0.5, 0.75, 1.0], labels=["Q1", "Q2", "Q3", "Q4"])
ergm_nodes["13Q"] = pd.qcut(ergm_nodes["13"], q=[0, 0.25, 0.5, 0.75, 1.0], labels=["Q1", "Q2", "Q3", "Q4"])
ergm_nodes["26Q"] = pd.qcut(ergm_nodes["26"], q=[0, 0.25, 0.5, 0.75, 1.0], labels=["Q1", "Q2", "Q3", "Q4"])
ergm_nodes["54Q"] = pd.qcut(ergm_nodes["54"], q=[0, 0.25, 0.5, 0.75, 1.0], labels=["Q1", "Q2", "Q3", "Q4"])

In [ ]:
ergm_nodes

,username,verified.type,2,4,13,26,54,2Q,4Q,13Q,26Q,54Q
0,@CDCgov,1,0.004235,0.002144,0.011127,0.001374,0.004126,Q4,Q4,Q4,Q4,Q4
1,@NBSaphierMD,1,0.060214,0.028441,0.001009,0.000711,0.000771,Q4,Q4,Q3,Q3,Q3
2,@hillsy191,0,0.045837,0.015824,0.001428,0.013830,0.001254,Q4,Q4,Q3,Q4,Q3
3,@cimtrbl2,0,0.000527,0.000528,0.015970,0.000521,0.030139,Q3,Q3,Q4,Q3,Q4
4,@JMCRHealthDept,1,0.006194,0.003581,0.006274,0.008779,0.005937,Q4,Q4,Q4,Q4,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...
23834,@georgesray,0,0.001078,0.001048,0.001071,0.001050,0.001014,Q3,Q3,Q3,Q3,Q3
23835,@middlefischer,0,0.001849,0.002175,0.002017,0.001905,0.001786,Q3,Q4,Q4,Q4,Q4
23836,@deniseparrish01,0,0.527144,0.011288,0.118513,0.003220,0.058992,Q4,Q4,Q4,Q4,Q4
23837,@LeslieStern12,0,0.000275,0.000282,0.000281,0.000278,0.000257,Q2,Q2,Q2,Q2,Q2


In [ ]:
ergm_nodes.columns

Index(['username', 'verified.type', '2', '4', '13', '26', '54', '2Q', '4Q',
       '13Q', '26Q', '54Q'],
      dtype='object')

In [ ]:
ergm_nodes = ergm_nodes[['username', 'verified.type','2Q', '4Q', '13Q', '26Q', '54Q']]
ergm_nodes

,username,verified.type,2Q,4Q,13Q,26Q,54Q
0,@CDCgov,1,Q4,Q4,Q4,Q4,Q4
1,@NBSaphierMD,1,Q4,Q4,Q3,Q3,Q3
2,@hillsy191,0,Q4,Q4,Q3,Q4,Q3
3,@cimtrbl2,0,Q3,Q3,Q4,Q3,Q4
4,@JMCRHealthDept,1,Q4,Q4,Q4,Q4,Q4
...,...,...,...,...,...,...,...
23834,@georgesray,0,Q3,Q3,Q3,Q3,Q3
23835,@middlefischer,0,Q3,Q4,Q4,Q4,Q4
23836,@deniseparrish01,0,Q4,Q4,Q4,Q4,Q4
23837,@LeslieStern12,0,Q2,Q2,Q2,Q2,Q2


In [ ]:
ergm_nodes.to_csv('ergm_nodes.csv')

### Trying the approach of 1/0 instead of floating point numbers:

If they're a top mask discussant then 1, else 0.

In [ ]:
df_top_maskers = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/df_top_users_final.csv')

In [ ]:
df_top_maskers['username'] = '@' + df_top_maskers['username'].astype(str)
df_top_maskers

In [ ]:
top_users = list(df_top_maskers['username'].unique())
top_users

In [ ]:
nodes = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/ergm_nodes_limited.csv')
nodes

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54
0,0,@CDCgov,1,verified,0.004235,0.002144,0.011127,0.001374,0.004126
1,1,@NBSaphierMD,1,verified,0.060214,0.028441,0.001009,0.000711,0.000771
2,3,@hillsy191,0,unverified,0.045837,0.015824,0.001428,0.013830,0.001254
3,5,@cimtrbl2,0,unverified,0.000527,0.000528,0.015970,0.000521,0.030139
4,6,@JMCRHealthDept,1,verified,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...,...,...,...,...
23834,83077,@georgesray,0,unverified,0.001078,0.001048,0.001071,0.001050,0.001014
23835,83121,@middlefischer,0,unverified,0.001849,0.002175,0.002017,0.001905,0.001786
23836,83130,@deniseparrish01,0,unverified,0.527144,0.011288,0.118513,0.003220,0.058992
23837,83172,@LeslieStern12,0,unverified,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
nodes = nodes[['username','2','4','13','26','54']]
nodes

,username,2,4,13,26,54
0,@CDCgov,0.004235,0.002144,0.011127,0.001374,0.004126
1,@NBSaphierMD,0.060214,0.028441,0.001009,0.000711,0.000771
2,@hillsy191,0.045837,0.015824,0.001428,0.013830,0.001254
3,@cimtrbl2,0.000527,0.000528,0.015970,0.000521,0.030139
4,@JMCRHealthDept,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...,...
23834,@georgesray,0.001078,0.001048,0.001071,0.001050,0.001014
23835,@middlefischer,0.001849,0.002175,0.002017,0.001905,0.001786
23836,@deniseparrish01,0.527144,0.011288,0.118513,0.003220,0.058992
23837,@LeslieStern12,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
nodes = nodes.rename(columns={'2': 'Topic2', '4': 'Topic4', '13': 'Topic13', '26': 'Topic26', '54': 'Topic54'})
nodes

,username,Topic2,Topic4,Topic13,Topic26,Topic54
0,@CDCgov,0.004235,0.002144,0.011127,0.001374,0.004126
1,@NBSaphierMD,0.060214,0.028441,0.001009,0.000711,0.000771
2,@hillsy191,0.045837,0.015824,0.001428,0.013830,0.001254
3,@cimtrbl2,0.000527,0.000528,0.015970,0.000521,0.030139
4,@JMCRHealthDept,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...,...
23834,@georgesray,0.001078,0.001048,0.001071,0.001050,0.001014
23835,@middlefischer,0.001849,0.002175,0.002017,0.001905,0.001786
23836,@deniseparrish01,0.527144,0.011288,0.118513,0.003220,0.058992
23837,@LeslieStern12,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
nodes[['Topic2','Topic4','Topic13','Topic26','Topic54']] = scaler.fit_transform(nodes[['Topic2','Topic4','Topic13','Topic26','Topic54']])
nodes

,username,Topic2,Topic4,Topic13,Topic26,Topic54
0,@CDCgov,0.004235,0.002144,0.011127,0.001374,0.004126
1,@NBSaphierMD,0.060214,0.028441,0.001009,0.000711,0.000771
2,@hillsy191,0.045837,0.015824,0.001428,0.013830,0.001254
3,@cimtrbl2,0.000527,0.000528,0.015970,0.000521,0.030139
4,@JMCRHealthDept,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...,...
23834,@georgesray,0.001078,0.001048,0.001071,0.001050,0.001014
23835,@middlefischer,0.001849,0.002175,0.002017,0.001905,0.001786
23836,@deniseparrish01,0.527144,0.011288,0.118513,0.003220,0.058992
23837,@LeslieStern12,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
df = nodes[['Topic2','Topic4','Topic13','Topic26','Topic54']]
df

,Topic2,Topic4,Topic13,Topic26,Topic54
0,0.004235,0.002144,0.011127,0.001374,0.004126
1,0.060214,0.028441,0.001009,0.000711,0.000771
2,0.045837,0.015824,0.001428,0.013830,0.001254
3,0.000527,0.000528,0.015970,0.000521,0.030139
4,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...
23834,0.001078,0.001048,0.001071,0.001050,0.001014
23835,0.001849,0.002175,0.002017,0.001905,0.001786
23836,0.527144,0.011288,0.118513,0.003220,0.058992
23837,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:

scaled_df['Topic2'] != nodes['Topic2']

0        False
1        False
2        False
3        False
4        False
         ...  
23834    False
23835    False
23836    False
23837    False
23838    False
Name: Topic2, Length: 23839, dtype: bool

In [ ]:
scaler.fit(df)
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)
scaled_df

,Topic2,Topic4,Topic13,Topic26,Topic54
0,0.004235,0.002144,0.011127,0.001374,0.004126
1,0.060214,0.028441,0.001009,0.000711,0.000771
2,0.045837,0.015824,0.001428,0.013830,0.001254
3,0.000527,0.000528,0.015970,0.000521,0.030139
4,0.006194,0.003581,0.006274,0.008779,0.005937
...,...,...,...,...,...
23834,0.001078,0.001048,0.001071,0.001050,0.001014
23835,0.001849,0.002175,0.002017,0.001905,0.001786
23836,0.527144,0.011288,0.118513,0.003220,0.058992
23837,0.000275,0.000282,0.000281,0.000278,0.000257


In [ ]:
for index, row in nodes.iterrows():
  author = str(row['username'])
  if author in top_users:
    nodes['masker'][index] = 1
  else:
    continue

<ipython-input-16-b036b6e98c7d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes['masker'][index] = 1


In [ ]:
#nodes.to_csv('nodes_masking.csv')

# Using top topic:

In [ ]:
nodes = pd.read_csv('nodes_avg_weights.csv')
nodes

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54
0,0,@CDCgov,1,verified,4.235155e-03,2.144329e-03,1.112680e-02,1.374142e-03,4.125985e-03
1,1,@NBSaphierMD,1,verified,6.021439e-02,2.844062e-02,1.009201e-03,7.112913e-04,7.714173e-04
2,2,@thenameis_ari,0,unverified,5.309836e-04,5.173393e-04,5.290470e-04,5.288367e-04,5.045694e-04
3,3,@hillsy191,0,unverified,4.583740e-02,1.582431e-02,1.427712e-03,1.382988e-02,1.253511e-03
4,4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
83252,83252,@SerraJovenich,0,unverified,1.720840e-03,1.654831e-03,1.704416e-03,1.752056e-03,1.613899e-03
83253,83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
83254,83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48
83255,83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
nodes['toptopic'] = nodes[['2','4','13','26','54']].idxmax(axis=1)

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54,toptopic
0,0,@CDCgov,1,verified,4.235155e-03,2.144329e-03,1.112680e-02,1.374142e-03,4.125985e-03,13
1,1,@NBSaphierMD,1,verified,6.021439e-02,2.844062e-02,1.009201e-03,7.112913e-04,7.714173e-04,2
2,2,@thenameis_ari,0,unverified,5.309836e-04,5.173393e-04,5.290470e-04,5.288367e-04,5.045694e-04,2
3,3,@hillsy191,0,unverified,4.583740e-02,1.582431e-02,1.427712e-03,1.382988e-02,1.253511e-03,2
4,4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2
...,...,...,...,...,...,...,...,...,...,...
83252,83252,@SerraJovenich,0,unverified,1.720840e-03,1.654831e-03,1.704416e-03,1.752056e-03,1.613899e-03,26
83253,83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2
83254,83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48,2
83255,83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2


In [ ]:
nodes['toptopic'] = 'Topic' + nodes['toptopic'].astype(str)

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54,toptopic
0,0,@CDCgov,1,verified,4.235155e-03,2.144329e-03,1.112680e-02,1.374142e-03,4.125985e-03,Topic13
1,1,@NBSaphierMD,1,verified,6.021439e-02,2.844062e-02,1.009201e-03,7.112913e-04,7.714173e-04,Topic2
2,2,@thenameis_ari,0,unverified,5.309836e-04,5.173393e-04,5.290470e-04,5.288367e-04,5.045694e-04,Topic2
3,3,@hillsy191,0,unverified,4.583740e-02,1.582431e-02,1.427712e-03,1.382988e-02,1.253511e-03,Topic2
4,4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Topic2
...,...,...,...,...,...,...,...,...,...,...
83252,83252,@SerraJovenich,0,unverified,1.720840e-03,1.654831e-03,1.704416e-03,1.752056e-03,1.613899e-03,Topic26
83253,83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Topic2
83254,83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48,Topic2
83255,83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Topic2


In [ ]:
nodes = nodes[['username','verified.type','toptopic']]
nodes

,username,verified.type,toptopic
0,@CDCgov,1,Topic13
1,@NBSaphierMD,1,Topic2
2,@thenameis_ari,0,Topic2
3,@hillsy191,0,Topic2
4,@Itsmoosej,0,Topic2
...,...,...,...
83252,@SerraJovenich,0,Topic26
83253,@rodsoutherland,0,Topic2
83254,@THEtacotina,0,Topic2
83255,@kaaaramarie,0,Topic2


In [ ]:
sentiment = pd.read_csv('nodes_quadrants_sentiment.csv')
sentiment

,Unnamed: 0,username,verified.type,Quad2,Quad4,Quad13,Quad26,Quad54,average_sentiment
0,0,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
1,7322,@NBSaphierMD,1,Q4,Q4,Q3,Q3,Q3,negative
2,7337,@hillsy191,0,Q4,Q4,Q3,Q4,Q3,negative
3,7357,@cimtrbl2,0,Q3,Q3,Q4,Q3,Q4,negative
4,7414,@JMCRHealthDept,1,Q4,Q4,Q4,Q4,Q4,negative
...,...,...,...,...,...,...,...,...,...
23834,138808,@georgesray,0,Q3,Q3,Q3,Q3,Q3,negative
23835,138810,@middlefischer,0,Q3,Q4,Q4,Q4,Q4,negative
23836,138812,@deniseparrish01,0,Q4,Q4,Q4,Q4,Q4,negative
23837,138813,@LeslieStern12,0,Q2,Q2,Q2,Q2,Q2,positive


In [ ]:
dfcombined = pd.merge(nodes,sentiment[['username','average_sentiment']],on='username', how='inner')
dfcombined

,username,verified.type,toptopic,average_sentiment
0,@CDCgov,1,Topic13,negative
1,@NBSaphierMD,1,Topic2,negative
2,@hillsy191,0,Topic2,negative
3,@cimtrbl2,0,Topic54,negative
4,@JMCRHealthDept,1,Topic26,negative
...,...,...,...,...
23834,@georgesray,0,Topic2,negative
23835,@middlefischer,0,Topic4,negative
23836,@deniseparrish01,0,Topic2,negative
23837,@LeslieStern12,0,Topic4,positive


In [ ]:
dfcombined.to_csv('nodes_toptopic_sentiment.csv')

# Normalizing data:

In [ ]:
from sklearn import preprocessing

In [ ]:
nodes = pd.read_csv('nodes_avg_weights.csv')
nodes

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54
0,0,@CDCgov,1,verified,4.235155e-03,2.144329e-03,1.112680e-02,1.374142e-03,4.125985e-03
1,1,@NBSaphierMD,1,verified,6.021439e-02,2.844062e-02,1.009201e-03,7.112913e-04,7.714173e-04
2,2,@thenameis_ari,0,unverified,5.309836e-04,5.173393e-04,5.290470e-04,5.288367e-04,5.045694e-04
3,3,@hillsy191,0,unverified,4.583740e-02,1.582431e-02,1.427712e-03,1.382988e-02,1.253511e-03
4,4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
83252,83252,@SerraJovenich,0,unverified,1.720840e-03,1.654831e-03,1.704416e-03,1.752056e-03,1.613899e-03
83253,83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
83254,83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48
83255,83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
cols = ['2','4','13','26','54']

# Overwrite old columns with transformed columns
nodes[cols] = preprocessing.normalize(nodes[cols], norm='l2', axis=1, copy=False)
nodes

,Unnamed: 0,username,verified.type,verified.label,2,4,13,26,54
0,0,@CDCgov,1,verified,3.294545e-01,1.668083e-01,8.655585e-01,1.068951e-01,3.209622e-01
1,1,@NBSaphierMD,1,verified,9.039976e-01,4.269785e-01,1.515112e-02,1.067860e-02,1.158127e-02
2,2,@thenameis_ari,0,unverified,4.546911e-01,4.430073e-01,4.530328e-01,4.528527e-01,4.320721e-01
3,3,@hillsy191,0,unverified,9.083659e-01,3.135925e-01,2.829317e-02,2.740685e-01,2.484099e-02
4,4,@Itsmoosej,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...
83252,83252,@SerraJovenich,0,unverified,4.553960e-01,4.379276e-01,4.510495e-01,4.636569e-01,4.270954e-01
83253,83253,@rodsoutherland,0,unverified,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
83254,83254,@THEtacotina,0,unverified,1.060428e-48,1.031578e-48,1.051735e-48,1.051725e-48,1.004456e-48
83255,83255,@kaaaramarie,0,unverified,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
nodes = nodes.drop(['verified.label'], axis=1)
nodes = nodes.drop(['Unnamed: 0'], axis=1)

In [ ]:
nodes[cols] = nodes[cols].round(5)
nodes

,username,verified.type,2,4,13,26,54
0,@CDCgov,1,0.32945,0.16681,0.86556,0.10690,0.32096
1,@NBSaphierMD,1,0.90400,0.42698,0.01515,0.01068,0.01158
2,@thenameis_ari,0,0.45469,0.44301,0.45303,0.45285,0.43207
3,@hillsy191,0,0.90837,0.31359,0.02829,0.27407,0.02484
4,@Itsmoosej,0,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...
83252,@SerraJovenich,0,0.45540,0.43793,0.45105,0.46366,0.42710
83253,@rodsoutherland,0,0.00000,0.00000,0.00000,0.00000,0.00000
83254,@THEtacotina,0,0.00000,0.00000,0.00000,0.00000,0.00000
83255,@kaaaramarie,0,1.00000,0.00000,0.00000,0.00000,0.00000


In [ ]:
nodes = nodes.rename(columns={'2': 'Topic2', '4': 'Topic4', '13': 'Topic13', '26': 'Topic26', '54': 'Topic54'})

In [ ]:
nodes.to_csv('nodes_normalized_5decimals.csv')

Turning it into 3 decimal points * 1000:

In [ ]:
nodes = pd.read_csv('nodes_normalized_5decimals.csv')
nodes = nodes.drop(['Unnamed: 0'], axis=1)
nodes

,username,verified.type,Topic2,Topic4,Topic13,Topic26,Topic54
0,@CDCgov,1,0.32945,0.16681,0.86556,0.10690,0.32096
1,@NBSaphierMD,1,0.90400,0.42698,0.01515,0.01068,0.01158
2,@thenameis_ari,0,0.45469,0.44301,0.45303,0.45285,0.43207
3,@hillsy191,0,0.90837,0.31359,0.02829,0.27407,0.02484
4,@Itsmoosej,0,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...
83252,@SerraJovenich,0,0.45540,0.43793,0.45105,0.46366,0.42710
83253,@rodsoutherland,0,0.00000,0.00000,0.00000,0.00000,0.00000
83254,@THEtacotina,0,0.00000,0.00000,0.00000,0.00000,0.00000
83255,@kaaaramarie,0,1.00000,0.00000,0.00000,0.00000,0.00000


In [ ]:
nodes.columns

Index(['username', 'verified.type', 'Topic2', 'Topic4', 'Topic13', 'Topic26',
       'Topic54'],
      dtype='object')

In [ ]:
cols = ['Topic2', 'Topic4', 'Topic13', 'Topic26','Topic54']
nodes[cols] = nodes[cols].round(3) * 1000

In [ ]:
nodes[cols] = nodes[cols].astype(int)
nodes

,username,verified.type,Topic2,Topic4,Topic13,Topic26,Topic54
0,@CDCgov,1,329,167,866,107,321
1,@NBSaphierMD,1,904,427,15,11,12
2,@thenameis_ari,0,455,443,453,453,432
3,@hillsy191,0,908,314,28,274,25
4,@Itsmoosej,0,0,0,0,0,0
...,...,...,...,...,...,...,...
83252,@SerraJovenich,0,455,438,451,464,427
83253,@rodsoutherland,0,0,0,0,0,0
83254,@THEtacotina,0,0,0,0,0,0
83255,@kaaaramarie,0,1000,0,0,0,0


In [ ]:
sentiment = pd.read_csv('nodes_quadrants_sentiment.csv')
sentiment = sentiment.drop(['Unnamed: 0'], axis=1)
sentiment

,username,verified.type,Quad2,Quad4,Quad13,Quad26,Quad54,average_sentiment
0,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
1,@NBSaphierMD,1,Q4,Q4,Q3,Q3,Q3,negative
2,@hillsy191,0,Q4,Q4,Q3,Q4,Q3,negative
3,@cimtrbl2,0,Q3,Q3,Q4,Q3,Q4,negative
4,@JMCRHealthDept,1,Q4,Q4,Q4,Q4,Q4,negative
...,...,...,...,...,...,...,...,...
23834,@georgesray,0,Q3,Q3,Q3,Q3,Q3,negative
23835,@middlefischer,0,Q3,Q4,Q4,Q4,Q4,negative
23836,@deniseparrish01,0,Q4,Q4,Q4,Q4,Q4,negative
23837,@LeslieStern12,0,Q2,Q2,Q2,Q2,Q2,positive


In [ ]:
dfcombined2 = pd.merge(nodes,dfcombined[['username','average_sentiment']],on='username', how='left')
dfcombined2 = dfcombined2.drop_duplicates(subset=['username'])
dfcombined2

,username,verified.type,Topic2,Topic4,Topic13,Topic26,Topic54,average_sentiment
0,@CDCgov,1,329,167,866,107,321,negative
7322,@NBSaphierMD,1,904,427,15,11,12,negative
7337,@thenameis_ari,0,455,443,453,453,432,negative
7340,@hillsy191,0,908,314,28,274,25,negative
7360,@Itsmoosej,0,0,0,0,0,0,positive
...,...,...,...,...,...,...,...,...
233289,@SerraJovenich,0,455,438,451,464,427,negative
233290,@rodsoutherland,0,0,0,0,0,0,negative
233291,@THEtacotina,0,0,0,0,0,0,positive
233292,@kaaaramarie,0,1000,0,0,0,0,negative


In [ ]:
dfcombined2 = dfcombined2.reset_index(drop=True)
dfcombined2.to_csv('nodes_integers_sentiment.csv')

#Since using this nodes file within the ERGM model allowed it to converge, let's try this 1/0 approach for all masking topics:

N.B.: The average topic weights here have to be 1 or 0. As in, if the user is in the top 90th percentile of those discussing topic X, then you will tag them as TopicX = 1;  otherwise, TopicX = 0. This will allow us to specify which of the topics get people to get more edges in this network.

In [ ]:
nodes = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/ergm_nodes_limited.csv')

In [ ]:
nodes = nodes[['username','verified.type']]
nodes

,username,verified.type
0,@CDCgov,1
1,@NBSaphierMD,1
2,@hillsy191,0
3,@cimtrbl2,0
4,@JMCRHealthDept,1
...,...,...
23834,@georgesray,0
23835,@middlefischer,0
23836,@deniseparrish01,0
23837,@LeslieStern12,0


In [ ]:
nodes = nodes.rename(columns={'verified.type': 'verified'})

In [ ]:
nodes['topic2'] = 0
nodes['topic4'] = 0
nodes['topic13'] = 0
nodes['topic26'] = 0
nodes['topic54'] = 0

In [ ]:
topic2 = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/topic2topusers.csv')
topic4 = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/topic4topusers.csv')
topic13 = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/topic13topusers.csv')
topic26 = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/topic26topusers.csv')
topic54 = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/topic54topusers.csv')

In [ ]:
topic2topusers = list(topic2['0'])
topic4topusers = list(topic4['0'])
topic13topusers = list(topic13['0'])
topic26topusers = list(topic26['0'])
topic54topusers = list(topic54['0'])

In [ ]:
for index, row in nodes.iterrows():
  author = str(row['username'])
  if author in topic2topusers:
    nodes['topic2'][index] = 1
  elif author in topic4topusers:
    nodes['topic4'][index] = 1
  elif author in topic13topusers:
    nodes['topic13'][index] = 1
  elif author in topic26topusers:
    nodes['topic26'][index] = 1
  elif author in topic54topusers:
    nodes['topic54'][index] = 1
  else:
    continue

In [ ]:
nodes

,username,verified,topic2,topic4,topic13,topic26,topic54
0,@CDCgov,1,0,0,1,0,0
1,@NBSaphierMD,1,1,0,0,0,0
2,@hillsy191,0,1,0,0,0,0
3,@cimtrbl2,0,0,0,1,0,0
4,@JMCRHealthDept,1,0,0,1,0,0
...,...,...,...,...,...,...,...
23834,@georgesray,0,0,0,0,0,0
23835,@middlefischer,0,0,0,0,0,0
23836,@deniseparrish01,0,1,0,0,0,0
23837,@LeslieStern12,0,0,0,0,0,0


In [ ]:
nodes.to_csv('nodes_top_maskers.csv')

## Using Flair Sentiment as Feature


In [ ]:
sentiment = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/ERGM-CDC/flair_avg_sentiment.csv')
sentiment

,author_id,average_sentiment,average_sentiment_num
0,146569971,negative,-0.226987
1,3065301110,negative,-0.333333
2,231338512,negative,-1.000000
3,183651112,negative,-0.800000
4,560791401,positive,1.000000
...,...,...,...
83246,1448769455782301696,negative,-1.000000
83247,27959855,negative,-1.000000
83248,355162109,positive,1.000000
83249,61066540,negative,-1.000000


In [ ]:
sentiment = sentiment[['author_id','average_sentiment']]
sentiment

,author_id,average_sentiment
0,146569971,negative
1,3065301110,negative
2,231338512,negative
3,183651112,negative
4,560791401,positive
...,...,...
83246,1448769455782301696,negative
83247,27959855,negative
83248,355162109,positive
83249,61066540,negative


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/tweetsconcat.csv')
df = df.drop(['index'], axis=1)
df

<ipython-input-17-bb0b51f21329>:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Research-Dr.Ammari/tweetsconcat.csv')


,entities,context_annotations,source,attachments,possibly_sensitive,text,public_metrics,reply_settings,id,lang,conversation_id,created_at,author_id,author,__twarc,referenced_tweets,in_reply_to_user_id,in_reply_to_user,geo,withheld
0,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344769054859534337'], 'med...",False,How do you keep an almost-eradicated virus in ...,"{'retweet_count': 36, 'reply_count': 19, 'like...",everyone,1344769057720070144,en,1344769057720070144,2020-12-31 22:15:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN
1,"{'hashtags': [{'start': 54, 'end': 62, 'tag': ...","[{'domain': {'id': '65', 'name': 'Interests an...",Sprout Social,"{'media_keys': ['3_1344760747851657219'], 'med...",False,Attending a small celebration? Take steps to m...,"{'retweet_count': 104, 'reply_count': 69, 'lik...",everyone,1344760752587071488,en,1344760752587071488,2020-12-31 21:42:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN
2,"{'hashtags': [{'start': 44, 'end': 52, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344741623805173764'], 'med...",False,An ongoing CDC study of household spread of #C...,"{'retweet_count': 68, 'reply_count': 33, 'like...",everyone,1344741626430816256,en,1344741626430816256,2020-12-31 20:26:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN
3,"{'hashtags': [{'start': 126, 'end': 134, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter Media Studio,"{'media_keys': ['13_1344705124866916352'], 'me...",False,How do we know if vaccines are safe? @aasian a...,"{'retweet_count': 38, 'reply_count': 34, 'like...",everyone,1344735073753387008,en,1344735073753387008,2020-12-31 20:00:00+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN
4,"{'hashtags': [{'start': 38, 'end': 46, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344725018392076288'], 'med...",False,New @CDCMMWR illustrates data showing #COVID19...,"{'retweet_count': 161, 'reply_count': 108, 'li...",everyone,1344725021198065664,en,1344725021198065664,2020-12-31 19:20:03+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233289,"{'annotations': [{'start': 81, 'end': 83, 'pro...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,NaN,False,The amount of people not wearing masks inside ...,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",everyone,1477069420358877184,en,1477069420358877184,2022-01-01 00:09:27+00:00,61066540,{'description': 'pediatric nurse. capricorn st...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,"{'place_id': '895f19a3c08d3d35', 'place_type':...",NaN
233290,"{'annotations': [{'start': 126, 'end': 128, 'p...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for iPhone,NaN,False,@DCOstrander74 @IssuesOfMyTime @TeacherBW @OhN...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,1477068786117197824,en,1477024855715287040,2022-01-01 00:06:55+00:00,1043938043844055040,{'description': 'Virginia Tech NHL Politics NA...,{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'type': 'replied_to', 'id': '147704417537165...",7.237729e+17,"{'description': '', 'profile_image_url': 'http...","{'place_id': '3f7a925ec706ea48', 'place_type':...",NaN
233291,"{'annotations': [{'start': 28, 'end': 55, 'pro...",NaN,Twitter for iPhone,NaN,False,BREAKING: the CDC says that Hexachor

In [ ]:
df2 = df["author"].astype('str')
df2 = df2.apply(lambda x: ast.literal_eval(x))
df2 = df2.apply(pd.Series)
df['username'] = pd.Series(df2['username'])
df

,entities,context_annotations,source,attachments,possibly_sensitive,text,public_metrics,reply_settings,id,lang,...,created_at,author_id,author,__twarc,referenced_tweets,in_reply_to_user_id,in_reply_to_user,geo,withheld,username
0,"{'hashtags': [{'start': 59, 'end': 67, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344769054859534337'], 'med...",False,How do you keep an almost-eradicated virus in ...,"{'retweet_count': 36, 'reply_count': 19, 'like...",everyone,1344769057720070144,en,...,2020-12-31 22:15:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
1,"{'hashtags': [{'start': 54, 'end': 62, 'tag': ...","[{'domain': {'id': '65', 'name': 'Interests an...",Sprout Social,"{'media_keys': ['3_1344760747851657219'], 'med...",False,Attending a small celebration? Take steps to m...,"{'retweet_count': 104, 'reply_count': 69, 'lik...",everyone,1344760752587071488,en,...,2020-12-31 21:42:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
2,"{'hashtags': [{'start': 44, 'end': 52, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344741623805173764'], 'med...",False,An ongoing CDC study of household spread of #C...,"{'retweet_count': 68, 'reply_count': 33, 'like...",everyone,1344741626430816256,en,...,2020-12-31 20:26:02+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
3,"{'hashtags': [{'start': 126, 'end': 134, 'tag'...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter Media Studio,"{'media_keys': ['13_1344705124866916352'], 'me...",False,How do we know if vaccines are safe? @aasian a...,"{'retweet_count': 38, 'reply_count': 34, 'like...",everyone,1344735073753387008,en,...,2020-12-31 20:00:00+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
4,"{'hashtags': [{'start': 38, 'end': 46, 'tag': ...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Sprout Social,"{'media_keys': ['3_1344725018392076288'], 'med...",False,New @CDCMMWR illustrates data showing #COVID19...,"{'retweet_count': 161, 'reply_count': 108, 'li...",everyone,1344725021198065664,en,...,2020-12-31 19:20:03+00:00,146569971,{'profile_image_url': 'https://pbs.twimg.com/p...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,CDCgov
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233289,"{'annotations': [{'start': 81, 'end': 83, 'pro...","[{'domain': {'id': '123', 'name': 'Ongoing New...",Twitter for iPhone,NaN,False,The amount of people not wearing masks inside ...,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",everyone,1477069420358877184,en,...,2022-01-01 00:09:27+00:00,61066540,{'description': 'pediatric nurse. capricorn st...,{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,"{'place_id': '895f19a3c08d3d35', 'place_type':...",NaN,kaaaramarie
233290,"{'annotations': [{'start': 126, 'end': 128, 'p...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",Twitter for iPhone,NaN,False,@DCOstrander74 @IssuesOfMyTime @TeacherBW @OhN...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,1477068786117197824,en,...,2022-01-01 00:06:55+00:00,1043938043844055040,{'description': 'Virginia Tech NHL Politics NA...,{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'type': 'replied_to', 'id': '147704417537165...",7.237729e+17,"{'description': '', 'profile_image_url': 'http...","{'place_id': '3f7a925ec706ea48', 'place_type':...",NaN,KevinMeador4
233291,"{'annotations': [{'start': 28, 'end': 55, 'pro...",NaN,Twitter for iPhone,NaN,False,BREAKING: the CDC says that Hexachordal Combin...,"{'retweet_count': 1, 'reply_count': 

In [ ]:
dfcombined = pd.merge(sentiment,df[['username','author_id']],on='author_id', how='inner')
dfcombined

,author_id,average_sentiment,username
0,146569971,negative,CDCgov
1,146569971,negative,CDCgov
2,146569971,negative,CDCgov
3,146569971,negative,CDCgov
4,146569971,negative,CDCgov
...,...,...,...
233289,1448769455782301696,negative,SerraJovenich
233290,27959855,negative,rodsoutherland
233291,355162109,positive,THEtacotina
233292,61066540,negative,kaaaramarie


In [ ]:
dfcombined = dfcombined[['average_sentiment', 'username']]

In [ ]:
dfcombined['username']= '@' + dfcombined['username'].astype(str)
dfcombined

<ipython-input-21-d84de0b6caac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcombined['username']= '@' + dfcombined['username'].astype(str)


,average_sentiment,username
0,negative,@CDCgov
1,negative,@CDCgov
2,negative,@CDCgov
3,negative,@CDCgov
4,negative,@CDCgov
...,...,...
233289,negative,@SerraJovenich
233290,negative,@rodsoutherland
233291,positive,@THEtacotina
233292,negative,@kaaaramarie


In [ ]:
nodes = pd.read_csv('nodes_ergm.csv')

In [ ]:
nodes = nodes.drop(['Unnamed: 0'], axis =1)
nodes

,username,verified.type,Quad2,Quad4,Quad13,Quad26,Quad54
0,@CDCgov,1,Q4,Q4,Q4,Q4,Q4
1,@NBSaphierMD,1,Q4,Q4,Q3,Q3,Q3
2,@hillsy191,0,Q4,Q4,Q3,Q4,Q3
3,@cimtrbl2,0,Q3,Q3,Q4,Q3,Q4
4,@JMCRHealthDept,1,Q4,Q4,Q4,Q4,Q4
...,...,...,...,...,...,...,...
23834,@georgesray,0,Q3,Q3,Q3,Q3,Q3
23835,@middlefischer,0,Q3,Q4,Q4,Q4,Q4
23836,@deniseparrish01,0,Q4,Q4,Q4,Q4,Q4
23837,@LeslieStern12,0,Q2,Q2,Q2,Q2,Q2


In [ ]:
dfcombined2 = pd.merge(nodes,dfcombined,on='username', how='inner')
dfcombined2

,username,verified.type,Quad2,Quad4,Quad13,Quad26,Quad54,average_sentiment
0,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
1,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
2,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
3,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
4,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
...,...,...,...,...,...,...,...,...
138810,@middlefischer,0,Q3,Q4,Q4,Q4,Q4,negative
138811,@middlefischer,0,Q3,Q4,Q4,Q4,Q4,negative
138812,@deniseparrish01,0,Q4,Q4,Q4,Q4,Q4,negative
138813,@LeslieStern12,0,Q2,Q2,Q2,Q2,Q2,positive


In [ ]:
dfcombined2 = dfcombined2.drop_duplicates(subset=['username'])
dfcombined2

,username,verified.type,Quad2,Quad4,Quad13,Quad26,Quad54,average_sentiment
0,@CDCgov,1,Q4,Q4,Q4,Q4,Q4,negative
7322,@NBSaphierMD,1,Q4,Q4,Q3,Q3,Q3,negative
7337,@hillsy191,0,Q4,Q4,Q3,Q4,Q3,negative
7357,@cimtrbl2,0,Q3,Q3,Q4,Q3,Q4,negative
7414,@JMCRHealthDept,1,Q4,Q4,Q4,Q4,Q4,negative
...,...,...,...,...,...,...,...,...
138808,@georgesray,0,Q3,Q3,Q3,Q3,Q3,negative
138810,@middlefischer,0,Q3,Q4,Q4,Q4,Q4,negative
138812,@deniseparrish01,0,Q4,Q4,Q4,Q4,Q4,negative
138813,@LeslieStern12,0,Q2,Q2,Q2,Q2,Q2,positive


In [ ]:
dfcombined2.to_csv('nodes_quadrants_sentiment.csv')

In [ ]:
# R model:
#
# lap_model <- ergm(snet ~ nodeocov("verified.type")
#                   + nodeicov("verified.type")
#                   + nodematch("Quad2",diff=T)
#                   + nodematch("Quad4",diff=T)
#                   + nodematch("Quad13",diff=T)
#                   + nodematch("Quad26",diff=T)
#                   + nodematch("Quad54",diff=T)
#                   + nodematch("average_sentiment",diff=T)
#                   + edges
#                   + mutual
# )